In [1]:
import pandas as pd
import numpy as np

# Section 7: Going Multidimensional

In this section, we'll kick things up a notch and work with multi-index dataframes. This allows us to support more than one level of labels, enabling us to reflect multidimensional datasets within the confines of a two-dimensional data structure.

**Hierarchichal indices** represent a hierarchy of relationships that become intricately coupled with our data/values. Using Pandas techniques and methods, we can very efficiently change these hierarchies in order to answer specific questions.

As a side note, multi-level index, multiindex, and hierarchical index are all used interchangeably within the Pandas community.

## Introducing New Data

In this section, we'll be working with a new dataset that contains daily stock information from 2014 through 2019 for the technology companies Apple, Facebook, Microsoft, Google, and Amazon.

In [2]:
tech_url = 'https://andybek.com/pandas-tech'

In [3]:
tech = pd.read_csv(tech_url)

In [4]:
tech.head()

,date,month,year,day,name,open,close,high,low,volume,volume_type
0,2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622,medium
1,2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851,low
2,2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719,medium
3,2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745,medium
4,2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246,medium


Here we see we have date in the first column, then columns of decomposed dates, the stock tickers under "name", the opening, closing, high, and low prices for that day, the volume of shares traded, and the qualitative trade volume.

Examining the shape of the dataframe, we have 7105 rows and 11 columns.

In [5]:
tech.shape

(7105, 11)

Let's also check out the type of data in the dataframe.

In [6]:
tech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7105 entries, 0 to 7104
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7105 non-null   object 
 1   month        7105 non-null   int64  
 2   year         7105 non-null   int64  
 3   day          7105 non-null   int64  
 4   name         7105 non-null   object 
 5   open         7105 non-null   float64
 6   close        7105 non-null   float64
 7   high         7105 non-null   float64
 8   low          7105 non-null   float64
 9   volume       7105 non-null   int64  
 10  volume_type  7105 non-null   object 
dtypes: float64(4), int64(4), object(3)
memory usage: 610.7+ KB


We have 4 floating point columns (open, close, high, low), 4 integer columns (month, year, day, volume), and three object columns (date, name, and volume_type).

## Index and RangeIndex

Let's review the terminology that we've grown accustomed to. In Series, the index is simply a label for each value in the series. In dataframes, the index still serves as a label for each row, but is accompanied by another set of labels for the column dimension.

Remember that an index of the type `RangeIndex`, which is simply an immutable or unchangable object that represents a series of increasing or decreasing integers. In Pandas, `RangeIndex` is further derived from the `index` class, which happens to be the same exact class type that the column labels inherit from

In [7]:
type(tech.index)

pandas.core.indexes.range.RangeIndex

In [8]:
type(tech.columns)

pandas.core.indexes.base.Index

So what is the "index" class? It's essentially another immutable data structure (in this case a Numpy array) that is ordered and sliceable.

Oftentimes we find the need to replace the default indices with something more meaningful. We've previously seen this with the `set_index()` method.

In [9]:
tech.set_index('date')

,month,year,day,name,open,close,high,low,volume,volume_type
date,,,,,,,,,,
2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622,medium
2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851,low
2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719,medium
2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745,medium
2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246,medium
...,...,...,...,...,...,...,...,...,...,...
2019-08-23,8,2019,23,MSFT,137.19,133.39,138.35,132.80,38515386,medium
2019-08-23,8,2019,23,AAPL,209.43,202.64,212.05,201.00,46882843,medium
2019-08-23,8,2019,23,GOOGL,1185.17,1153.58,1195.67,1150.00,1813141,medium


But what does it mean for *date* to "be" an index? One of the key implications has to do with how we select from the dataframe. If we wanted to extract or index all the prices for August 1, 2019, we can use that for our selection.

In [10]:
tech.set_index('date').loc['2019-08-01']

,month,year,day,name,open,close,high,low,volume,volume_type
date,,,,,,,,,,
2019-08-01,8,2019,1,GOOGL,1217.63,1211.78,1236.30,1207.00,1771271,medium
2019-08-01,8,2019,1,FB,194.17,192.73,198.47,190.88,17777013,medium
2019-08-01,8,2019,1,MSFT,137.00,138.06,140.94,136.93,40557502,medium
2019-08-01,8,2019,1,AMZN,1871.72,1855.32,1897.92,1844.01,4713311,medium
2019-08-01,8,2019,1,AAPL,213.90,208.43,218.03,206.74,54017922,medium


## Creating a MultiIndex

In the previous lecture, we used the `set_index()` method to change the index from the default to something more meaningful. We can take this a set further and use more than one field as the index for our dataframe.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

Let's start simply: instead of "date", we'll pass in a list of strings on which to set indices. 

In [11]:
tech.set_index(['date','name'])

month  year  day  ...      low    volume  volume_type
date       name                     ...                                
2014-01-02 FB         1  2014    2  ...    54.19  43257622       medium
           AAPL       1  2014    2  ...    78.86   8398851          low
           GOOGL      1  2014    2  ...   554.13   1822719       medium
           MSFT       1  2014    2  ...    37.10  30643745       medium
           AMZN       1  2014    2  ...   394.02   2140246       medium
...                 ...   ...  ...  ...      ...       ...          ...
2019-08-23 MSFT       8  2019   23  ...   132.80  38515386       medium
           AAPL       8  2019   23  ...   201.00  46882843       medium
           GOOGL      8  2019   23  ...  1150.00   1813141       medium
           AMZN       8  2019   23  ...  1745.23   5277898       medium
           FB         8  2019   23  ...   176.66  17331221       medium

[7105 rows x 9 columns]

The result is a MultiIndex, in which a single index has more than one component to it. When we promoted the "date" and "name" columns to indices, they were removed as regular columns from the dataframe. 

This looks a bit weird initially. But we'll get used to it.

Let's first set the `inplace` parameter to `True`.

In [12]:
tech.set_index(['date','name'], inplace = True)

Now let's look at the first two days.

In [13]:
tech.head(10)

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium
2014-01-03 FB         1  2014    3  ...   54.53  38287706       medium
           GOOGL      1  2014    3  ...  552.47   1669229       medium
           MSFT       1  2014    3  ...   36.60  31134795       medium
           AAPL       1  2014    3  ...   77.20  14043410          low
           AMZN       1  2014    3  ...  396.22   2213512       medium

[10 rows x 9 columns]

In our dataframe, we see that there are two bold-faced columns: the date and the name of the stock. The date is no longer repeating, where blank spaces indicate the data from above is carried on. 

Hierarchical indexing is not only about looks, but by using a multiindex, we're creating a hierarchy of relationships within our data where the information across the two index levels is inseparable from the actual values.

Let's see what type of object this is.


In [14]:
type(tech.index)

pandas.core.indexes.multi.MultiIndex

We see that our index is now of the type "MultiIndex". Cool!

## MultiIndex from `read_csv()`

It turns out that we don't have to wait until the dataframe is read in to set a multiindex. Instead, we can set the multiindex (or even a single index for that matter) when reading in the dataframe with `read_csv`!

All we really need to do is pass in a list ofthe index columns to the `index_col` parameter.

In [15]:
pd.read_csv(tech_url, index_col = ['date','name'])

month  year  day  ...      low    volume  volume_type
date       name                     ...                                
2014-01-02 FB         1  2014    2  ...    54.19  43257622       medium
           AAPL       1  2014    2  ...    78.86   8398851          low
           GOOGL      1  2014    2  ...   554.13   1822719       medium
           MSFT       1  2014    2  ...    37.10  30643745       medium
           AMZN       1  2014    2  ...   394.02   2140246       medium
...                 ...   ...  ...  ...      ...       ...          ...
2019-08-23 MSFT       8  2019   23  ...   132.80  38515386       medium
           AAPL       8  2019   23  ...   201.00  46882843       medium
           GOOGL      8  2019   23  ...  1150.00   1813141       medium
           AMZN       8  2019   23  ...  1745.23   5277898       medium
           FB         8  2019   23  ...   176.66  17331221       medium

[7105 rows x 9 columns]

There are other ways create multiindex dataframes as well, but we won't cover them here. Generally speaking, those other methods should only be utilized for very specific cases that necessitate them.



## Indexing Hierarchical DataFrames

So we've set hierarchical dataframes. Now how do we extract values from them? When we set the multiindex, we created an association between each pair of date and stock ticker and a value in our dataframe. Thus, indexing the data will look a bit different.

In [16]:
tech.head(10)

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium
2014-01-03 FB         1  2014    3  ...   54.53  38287706       medium
           GOOGL      1  2014    3  ...  552.47   1669229       medium
           MSFT       1  2014    3  ...   36.60  31134795       medium
           AAPL       1  2014    3  ...   77.20  14043410          low
           AMZN       1  2014    3  ...  396.22   2213512       medium

[10 rows x 9 columns]

Suppose we want to know what price Google closed at on January 2, 2014. Let's first try selecting for that particular date.

In [17]:
tech.loc['2014-01-02']

,month,year,day,open,close,high,low,volume,volume_type
name,,,,,,,,,
FB,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
AAPL,1,2014,2,79.38,79.02,79.58,78.86,8398851,low
GOOGL,1,2014,2,557.73,556.56,558.88,554.13,1822719,medium
MSFT,1,2014,2,37.35,37.16,37.40,37.10,30643745,medium
AMZN,1,2014,2,398.80,397.97,399.36,394.02,2140246,medium


Now we've isolated the full dataset for that date. To avoid being redundant, Pandas automatically drops the index that we are indexing along (in this case, "date").

Next, let's extract the Google data by going back to our `loc[]` indexer and adding another label to it.

In [18]:
tech.loc['2014-01-02', 'GOOGL']

month                1
year              2014
day                  2
open            557.73
close           556.56
high            558.88
low             554.13
volume         1822719
volume_type     medium
Name: (2014-01-02, GOOGL), dtype: object

What returns is a Series containing the values for Google on that single date. Now to isolate to close price, we can simply grab it out of the series as an attribute.

In [19]:
tech.loc['2014-01-02', 'GOOGL'].close

556.56

Another perhaps more elegant way of doing the same thing is to take advantage of the "type coupling" that we have between the "date" and "name" fields. They are both different levels in our multiindex, and so we can treat that as one dimension and capture it in a single tuple of values. In other words, we can identify the rows by the multiindex in one go by using a tuple, instead of performing this task iteratively.  By using a tuple, we are able to pass in multiple index labels while remaining in the first dimension (rows) of the `loc[]` indexer.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

In [20]:
tech.loc[('2014-01-02', 'GOOGL')]

month                1
year              2014
day                  2
open            557.73
close           556.56
high            558.88
low             554.13
volume         1822719
volume_type     medium
Name: (2014-01-02, GOOGL), dtype: object

Best of all, we can go in and make use of the second dimension (columns) within `loc[]`.

In [21]:
tech.loc[('2014-01-02', 'GOOGL'), 'close']

556.56

What about selecting by position using `iloc[]`? This actually works the exact same way in multiindex dataframs as it does in single index dataframes. The hierarchical structure has no impact on how `iloc[]` functions.

In [22]:
tech.head()

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium

[5 rows x 9 columns]

For example, on January 2, 2014 Google is at index position 2 (index starting at 0) in the dataframe, with the closing price as column position 4 (again index starting at 0). To get that value, we just use `iloc[]` the way we always have.

In [23]:
tech.iloc[2,4]

556.56

As another example, let's select the opening and closing prices for Apple on January 3, 2021.

In [24]:
tech.head(10)

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium
2014-01-03 FB         1  2014    3  ...   54.53  38287706       medium
           GOOGL      1  2014    3  ...  552.47   1669229       medium
           MSFT       1  2014    3  ...   36.60  31134795       medium
           AAPL       1  2014    3  ...   77.20  14043410          low
           AMZN       1  2014    3  ...  396.22   2213512       medium

[10 rows x 9 columns]

The label-based approach with `loc[]` would be as follows:

In [25]:
tech.loc[('2014-01-03', 'AAPL'), ['open', 'close']]

open        79
close    77.28
Name: (2014-01-03, AAPL), dtype: object

The position-based approach with `iloc[]` would be:

In [26]:
tech.iloc[8, [3,4]]

open        79
close    77.28
Name: (2014-01-03, AAPL), dtype: object

The result is exactly the same!

## Indexing Ranges and Slices and the `slice()` Object in MultiIndex DataFrames

We previously looked at ways to select values from multiindex dataframes by label or position. Now we'll go further and extract slices.

Suppose we want to extract multiple days from our tech stocks dataframe. If we want to select multiple dates, we just pass in a list of dates that we want to select. In the command below, the list of dates corresponds to the outer level of the index (the "date" level).

In [27]:
tech.loc[['2015-01-06', '2015-01-07']]

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2015-01-06 AAPL       1  2015    6  ...  104.63  65797116       medium
           FB         1  2015    6  ...   75.36  27399288       medium
           AMZN       1  2015    6  ...  292.38   3519034       medium
           MSFT       1  2015    6  ...   45.54  36447854       medium
           GOOGL      1  2015    6  ...  505.55   2731813       medium
2015-01-07 FB         1  2015    7  ...   75.82  22045333       medium
           MSFT       1  2015    7  ...   45.49  29114061       medium
           GOOGL      1  2015    7  ...  503.65   2345875       medium
           AMZN       1  2015    7  ...  295.33   2640349       medium
           AAPL       1  2015    7  ...  106.70  40105934       medium

[10 rows x 9 columns]

If we also want a subset of stock names, we just need to specify those in another list of index labels. However, this must be done within a tuple. If we do not do this, Pandas will think we are indexing along the column axis.

In [28]:
## This will not work
# tech.loc[['2015-01-06', '2015-01-07'], ['FB', 'AMZN']]

To get this to work, we have to wrap the two lists of indexes (dates and stock names) into a tuple. On the outer level (column axis), let's start by selecting all columns by using a colon `:`.

In [29]:
tech.loc[(['2015-01-06', '2015-01-07'], ['FB', 'AMZN']), :]

month  year  day    open  ...    high     low    volume  volume_type
date       name                            ...                                       
2015-01-06 FB        1  2015    6   77.23  ...   77.59   75.36  27399288       medium
           AMZN      1  2015    6  302.20  ...  303.00  292.38   3519034       medium
2015-01-07 FB        1  2015    7   76.76  ...   77.36   75.82  22045333       medium
           AMZN      1  2015    7  297.54  ...  301.28  295.33   2640349       medium

[4 rows x 9 columns]

If we only want to select particular columns, we can easily do so by passing in a list of columns labels!

In [30]:
tech.loc[(['2015-01-06', '2015-01-07'], ['FB', 'AMZN']), ['close', 'volume']]

close    volume
date       name                  
2015-01-06 FB     76.15  27399288
           AMZN  295.29   3519034
2015-01-07 FB     76.15  22045333
           AMZN  298.42   2640349

We can also slice our multidimensional dataframe by specifying a range of values separated by a colon. For instance, perhaps we want to select a range of dates from the outer level of our multiindex, and a range of columns.

In [31]:
tech.loc['2017-01-03':'2017-01-31','open':"low"]

open   close    high     low
date       name                                 
2017-01-03 AMZN   757.92  753.67  758.76  747.70
           FB     116.03  116.86  117.84  115.51
           MSFT    62.79   62.58   62.84   62.13
           AAPL   115.80  116.15  116.33  114.76
           GOOGL  800.62  808.01  811.44  796.89
...                  ...     ...     ...     ...
2017-01-31 MSFT    64.86   64.65   65.15   64.26
           AAPL   121.15  121.35  121.39  120.62
           FB     130.17  130.32  130.66  129.52
           GOOGL  819.50  820.19  823.07  813.40
           AMZN   823.75  823.48  826.99  819.56

[100 rows x 4 columns]

But what if we wanted to go a step further and isolate a specific stock (e.g. Google) within this date range slice? We can do that too. But how?

Based on what we've done already, the first thing to try might be to wrap the index dimension in a tuple and add the stock as the second item in the tuple. 

Unfortunately, this does not quite work:

In [32]:
## This results in a syntax error:
# tech.loc[('2017-01-03':'2017-01-31', 'GOOGL'),'open':"low"]

So what do we do? Well, in order to slice on a hierarchical index, we have to use the `slice()` object. In order to isolate Google stock prices in the sliced date range, we have to wrap our date range within a slice object, then wrap that in a tuple together with "GOOGL".

Importantly, we do not use the colon `:` slice operator. Instead, we identify the start and end of the slice within the `slice()` object, separated by a column.



In [33]:
tech.loc[(slice('2017-01-03','2017-01-31'), 'GOOGL'),'open':"low"]

,,open,close,high,low
date,name,,,,
2017-01-03,GOOGL,800.62,808.01,811.44,796.89
2017-01-04,GOOGL,809.89,807.77,813.43,804.11
2017-01-05,GOOGL,807.50,813.02,813.74,805.92
2017-01-06,GOOGL,814.99,825.21,828.96,811.50
2017-01-09,GOOGL,826.37,827.18,830.43,821.62
2017-01-10,GOOGL,827.07,826.01,829.41,823.14
2017-01-11,GOOGL,826.62,829.86,829.90,821.47
2017-01-12,GOOGL,828.38,829.53,830.38,821.01
2017-01-13,GOOGL,831.00,830.94,834.65,829.52


Let's do another example, in which we want the opening prices for Facebook and Amazon for all the dates in the dataframe. In other words, since we want the data for all of the dates, we need to skip the outer level of the index. But within the inner label, we want FB and AMZN only, and on the column axis we want "open" only.

Let's first try the most intuitive solution, which is to open a tuple for the multiindex, use a colon to select all dates from the outer dimension of the index, and identify FB and AMZN in a list as the inner dimension of the index. As you might have guessed, this does not work. 

In [34]:
## This results in a syntax error
# tech.loc[(:, ["FB",'AMZN']), 'open']

To get this to work, we once again have to invoke the `slice()` object. In order to slice for everything in a given dimension, we use `None`.

In [35]:
tech.loc[(slice(None), ['FB','AMZN']), "open"]

date        name
2014-01-02  FB        54.86
            AMZN     398.80
2014-01-03  FB        55.00
            AMZN     398.29
2014-01-06  AMZN     396.13
                     ...   
2019-08-21  AMZN    1819.39
2019-08-22  FB       183.43
            AMZN    1828.00
2019-08-23  AMZN    1793.03
            FB       180.84
Name: open, Length: 2842, dtype: float64

## BONUS - Use Colons `:` with `pd.IndexSlice`

In the previous lecture, we saw how indexing slices when selecting along multiple indices requires comma-separated slice boundaries within a `slice()` object.

There is an alternative approach that allows us to use the colon operator instead. We can do this with the `pd.IndexSlice` object.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html

Suppose we want the high and low prices for all trading days in the dataset for Amazon and Facebook. We can achieve this by using `pd.IndexSlice[]`, which is an indexor object that uses square brackets. It allows us to slice multiindexes more easily and intuitively using the colon `:` operator, without the need to invoke the `slice()` object

In [36]:
tech.loc[pd.IndexSlice[:, ['FB','AMZN']], ['high','low']]

high      low
date       name                  
2014-01-02 FB      55.22    54.19
           AMZN   399.36   394.02
2014-01-03 FB      55.65    54.53
           AMZN   402.71   396.22
2014-01-06 AMZN   397.00   388.42
...                  ...      ...
2019-08-21 AMZN  1829.58  1815.00
2019-08-22 FB     184.11   179.91
           AMZN  1829.41  1800.10
2019-08-23 AMZN  1804.90  1745.23
           FB     183.13   176.66

[2842 rows x 2 columns]

When working with the more complex multiindexes, and we need to call `pd.IndexSlice[]` several times, it is a good idea to assign it to a shorter variable so you don't have to keep typing the command.

In [37]:
i = pd.IndexSlice
tech.loc[i[:, 'FB'], ['high','low']]

,,high,low
date,name,,
2014-01-02,FB,55.22,54.19
2014-01-03,FB,55.65,54.53
2014-01-06,FB,57.26,54.05
2014-01-07,FB,58.55,57.22
2014-01-08,FB,58.41,57.23
...,...,...,...
2019-08-19,FB,187.50,184.85
2019-08-20,FB,186.00,182.39
2019-08-21,FB,185.90,183.14


Let's try another example, where we want a week's worth of data from January 6 through January 10 in 2014 for FB and AMZN, high and low prices. The `pd.IndexSlice[]` selector allows us to use colons to make those selections.
* Also notice how we don't need to use tuples for this.

In [38]:
tech.loc[i['2014-01-06':'2014-01-10', ["FB","AMZN"]], ["high","low"]]

high     low
date       name                
2014-01-06 AMZN  397.00  388.42
           FB     57.26   54.05
2014-01-07 FB     58.55   57.22
           AMZN  398.47  394.29
2014-01-08 AMZN  403.00  396.04
           FB     58.41   57.23
2014-01-09 AMZN  406.89  398.44
           FB     58.96   56.65
2014-01-10 FB     58.30   57.06
           AMZN  403.76  393.80

## Cross Sections with `xs()`

Many of these multiindexing methods are powerful and useful, but not particularly intuitive.

The `xs()` method is a subset of label-based indexing, but has a much more straightforward syntax. 
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html

Suppose we want a cross-section of our dataframe for the first trading day of the year 2019. We could do the following.

In [39]:
tech.xs('2019-01-02')

,month,year,day,open,close,high,low,volume,volume_type
name,,,,,,,,,
MSFT,1,2019,2,99.55,101.12,101.75,98.94,35329345,medium
FB,1,2019,2,128.99,135.68,137.51,128.56,28146193,medium
GOOGL,1,2019,2,1027.20,1054.68,1060.79,1025.28,1593395,medium
AMZN,1,2019,2,1465.20,1539.13,1553.36,1460.93,7983103,medium
AAPL,1,2019,2,154.89,157.92,158.85,154.23,37039737,medium


At first there's nothing particularly impressive here. There's not much difference between this and using the `loc[]` indexer.

In [40]:
tech.loc['2019-01-02']

,month,year,day,open,close,high,low,volume,volume_type
name,,,,,,,,,
MSFT,1,2019,2,99.55,101.12,101.75,98.94,35329345,medium
FB,1,2019,2,128.99,135.68,137.51,128.56,28146193,medium
GOOGL,1,2019,2,1027.20,1054.68,1060.79,1025.28,1593395,medium
AMZN,1,2019,2,1465.20,1539.13,1553.36,1460.93,7983103,medium
AAPL,1,2019,2,154.89,157.92,158.85,154.23,37039737,medium


But what if we wanted to skip the date level within our index and select solely based on stock name? If we wanted to do this with `loc()`, we'd need to use a `slice()` object within a tuple, as we saw earlier.


In [41]:
tech.loc[(slice(None), 'FB'), :]

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2014-01-02,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
2014-01-03,FB,1,2014,3,55.00,54.56,55.65,54.53,38287706,medium
2014-01-06,FB,1,2014,6,54.39,57.20,57.26,54.05,68974359,high
2014-01-07,FB,1,2014,7,57.67,57.92,58.55,57.22,77329009,high
2014-01-08,FB,1,2014,8,57.59,58.23,58.41,57.23,56800776,high
...,...,...,...,...,...,...,...,...,...,...
2019-08-19,FB,8,2019,19,186.01,186.17,187.50,184.85,9699661,low
2019-08-20,FB,8,2019,20,185.45,183.81,186.00,182.39,10087592,low
2019-08-21,FB,8,2019,21,185.00,183.55,185.90,183.14,8409548,low


But with `xs()`, we simply specify our `level` (that is, the value that we're indexing by), and pass in the label that you want to select by. In this case, the company name is at level 1, and the date is at level 0.

In [42]:
tech.xs('FB', level = 1)

,month,year,day,open,close,high,low,volume,volume_type
date,,,,,,,,,
2014-01-02,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
2014-01-03,1,2014,3,55.00,54.56,55.65,54.53,38287706,medium
2014-01-06,1,2014,6,54.39,57.20,57.26,54.05,68974359,high
2014-01-07,1,2014,7,57.67,57.92,58.55,57.22,77329009,high
2014-01-08,1,2014,8,57.59,58.23,58.41,57.23,56800776,high
...,...,...,...,...,...,...,...,...,...
2019-08-19,8,2019,19,186.01,186.17,187.50,184.85,9699661,low
2019-08-20,8,2019,20,185.45,183.81,186.00,182.39,10087592,low
2019-08-21,8,2019,21,185.00,183.55,185.90,183.14,8409548,low


Note that the output is technically not identical to the output when using the `loc[]` indexer. By default, `xs()` drops the level that we're selecting from, in this case the "name" index. But you can override this behavior by setting the `drop_level` parameter to `False`.

In [43]:
tech.xs('FB', level = 1, drop_level = False)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2014-01-02,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
2014-01-03,FB,1,2014,3,55.00,54.56,55.65,54.53,38287706,medium
2014-01-06,FB,1,2014,6,54.39,57.20,57.26,54.05,68974359,high
2014-01-07,FB,1,2014,7,57.67,57.92,58.55,57.22,77329009,high
2014-01-08,FB,1,2014,8,57.59,58.23,58.41,57.23,56800776,high
...,...,...,...,...,...,...,...,...,...,...
2019-08-19,FB,8,2019,19,186.01,186.17,187.50,184.85,9699661,low
2019-08-20,FB,8,2019,20,185.45,183.81,186.00,182.39,10087592,low
2019-08-21,FB,8,2019,21,185.00,183.55,185.90,183.14,8409548,low


But wait, there is more to this method! For instance, we could select from multiple levels of our index by passing same-sized **tuples** to both the `key` and `level` parameters.

In [44]:
tech.xs(key=('2019-01-02', 'FB'), level = (0, 1), drop_level = False)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2019-01-02,FB,1,2019,2,128.99,135.68,137.51,128.56,28146193,medium


When we get to multi-index columns later on, we'll see that we can use the exact same method to select from the other axis by setting the `axis` parameter to 1.

A quick note is that slicing using `xs()` requires the use of the `slice()` object. Colons are not functional.

## Skill Challenge

#### 1. From the *tech* dataframe, select all of the stock prices between July 13, 2015 and August 17, 2016. Assign the resulting dataframe slice to the variable *tech_df2*.

This can be accomplished using `pd.IndexSlice` as follows:

In [45]:
tech_df2 = tech.loc[pd.IndexSlice['2015-07-13':'2016-08-17']]

In [46]:
tech_df2

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2015-07-13 MSFT       7  2015   13  ...   44.95  28178329       medium
           AMZN       7  2015   13  ...  447.54   3956802       medium
           FB         7  2015   13  ...   88.42  29976670       medium
           GOOGL      7  2015   13  ...  558.70   2089641       medium
           AAPL       7  2015   13  ...  124.32  41440538       medium
...                 ...   ...  ...  ...     ...       ...          ...
2016-08-17 GOOGL      8  2016   17  ...  796.30   1066070       medium
           AMZN       8  2016   17  ...  759.20   1891116          low
           MSFT       8  2016   17  ...   57.23  18856423       medium
           FB         8  2016   17  ...  122.85  13794179          low
           AAPL       8  2016   17  ...  108.34  25355976       medium

[1395 rows x 9 columns]

Note that we could have also done this just using the `loc[]` indexer.

In [47]:
tech.loc['2015-07-13':'2016-08-17']

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2015-07-13 MSFT       7  2015   13  ...   44.95  28178329       medium
           AMZN       7  2015   13  ...  447.54   3956802       medium
           FB         7  2015   13  ...   88.42  29976670       medium
           GOOGL      7  2015   13  ...  558.70   2089641       medium
           AAPL       7  2015   13  ...  124.32  41440538       medium
...                 ...   ...  ...  ...     ...       ...          ...
2016-08-17 GOOGL      8  2016   17  ...  796.30   1066070       medium
           AMZN       8  2016   17  ...  759.20   1891116          low
           MSFT       8  2016   17  ...   57.23  18856423       medium
           FB         8  2016   17  ...  122.85  13794179          low
           AAPL       8  2016   17  ...  108.34  25355976       medium

[1395 rows x 9 columns]

#### 2. Select 10 days are random from *tech_df2*, but only AAPL price data.

To accomplish this, we first select only AAPL price data from *tech_df2*, and then we can use chain on the `sample()` method to randomly grab 10 dates.

In [48]:
tech_df2.xs(key = "AAPL", level = 1, drop_level = False).sample(10)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2016-08-15,AAPL,8,2016,15,108.14,109.48,109.54,108.08,25868209,medium
2016-06-14,AAPL,6,2016,14,97.32,97.46,98.48,96.75,31931944,medium
2015-09-01,AAPL,9,2015,1,110.15,107.72,111.88,107.36,76845860,high
2016-04-08,AAPL,4,2016,8,108.91,108.66,109.77,108.17,23581740,medium
2015-11-27,AAPL,11,2015,27,118.29,117.81,118.41,117.60,13046445,low
2016-01-25,AAPL,1,2016,25,101.52,99.44,101.53,99.21,51794525,medium
2016-06-29,AAPL,6,2016,29,93.97,94.40,94.55,93.63,36531006,medium
2016-08-02,AAPL,8,2016,2,106.05,104.48,106.07,104.00,33816556,medium
2015-09-14,AAPL,9,2015,14,116.58,115.31,116.89,114.86,58363431,medium


Alternatively using the `slice()` object:

In [49]:
tech_df2.loc[(slice(None), 'AAPL'), :].sample(10)

,,month,year,day,open,close,high,low,volume,volume_type
date,name,,,,,,,,,
2016-01-26,AAPL,1,2016,26,99.93,99.99,100.88,98.07,75077002,high
2015-10-21,AAPL,10,2015,21,114.00,113.76,115.58,113.70,42326974,medium
2015-09-17,AAPL,9,2015,17,115.66,113.92,116.49,113.72,64112641,medium
2015-12-09,AAPL,12,2015,9,117.64,115.62,117.69,115.08,46361357,medium
2015-12-17,AAPL,12,2015,17,112.02,108.98,112.25,108.98,44772827,medium
2016-04-05,AAPL,4,2016,5,109.51,109.81,110.73,109.42,26578652,medium
2016-02-17,AAPL,2,2016,17,96.67,98.12,98.21,96.15,44863243,medium
2015-12-08,AAPL,12,2015,8,117.52,118.23,118.60,116.86,34309450,medium
2015-07-24,AAPL,7,2015,24,125.32,124.50,125.74,123.90,42162332,medium


#### 3. Select all of the intraday *high* and *low* prices for AAPL and GOOGL in all of the dates in *tech_df2*.

We can achieve this by using `loc[]` to select only AAPL and GOOGL from the "name" index level, high and low from the columns.

In [50]:
tech_df2.loc[(slice(None), ["AAPL", "GOOGL"]), ["low", "high"]]

low    high
date       name                 
2015-07-13 GOOGL  558.70  572.85
           AAPL   124.32  125.76
2015-07-14 GOOGL  574.17  589.71
           AAPL   125.04  126.37
2015-07-15 GOOGL  580.21  588.69
...                  ...     ...
2016-08-15 AAPL   108.08  109.54
2016-08-16 GOOGL  797.00  804.26
           AAPL   109.21  110.23
2016-08-17 GOOGL  796.30  805.63
           AAPL   108.34  109.37

[558 rows x 2 columns]

Alternatively using `pd.IndexSlice[]`

In [51]:
tech_df2.loc[pd.IndexSlice[:, ['AAPL', 'GOOGL']], ['low','high']]

low    high
date       name                 
2015-07-13 GOOGL  558.70  572.85
           AAPL   124.32  125.76
2015-07-14 GOOGL  574.17  589.71
           AAPL   125.04  126.37
2015-07-15 GOOGL  580.21  588.69
...                  ...     ...
2016-08-15 AAPL   108.08  109.54
2016-08-16 GOOGL  797.00  804.26
           AAPL   109.21  110.23
2016-08-17 GOOGL  796.30  805.63
           AAPL   108.34  109.37

[558 rows x 2 columns]

## The Anatomy of the MultiIndex Object

How is a multiindex object structurally put together in Pandas? We know that a dataframe has a multi-level index. But it's not obvious from just looking at the dataframe itself.

In [52]:
tech.head()

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium

[5 rows x 9 columns]

But from its type, there's nothing obvious that points it out as a multiindex dataframe.

In [53]:
type(tech)

pandas.core.frame.DataFrame

If we look at the type of the index, we do see that it's a MultiIndex dataframe.

In [54]:
type(tech.index)

pandas.core.indexes.multi.MultiIndex

Let's take a closer look at the innards of the multiindex

In [55]:
tech.index

MultiIndex([('2014-01-02',    'FB'),
            ('2014-01-02',  'AAPL'),
            ('2014-01-02', 'GOOGL'),
            ('2014-01-02',  'MSFT'),
            ('2014-01-02',  'AMZN'),
            ('2014-01-03',    'FB'),
            ('2014-01-03', 'GOOGL'),
            ('2014-01-03',  'MSFT'),
            ('2014-01-03',  'AAPL'),
            ('2014-01-03',  'AMZN'),
            ...
            ('2019-08-22',  'MSFT'),
            ('2019-08-22',    'FB'),
            ('2019-08-22',  'AMZN'),
            ('2019-08-22',  'AAPL'),
            ('2019-08-22', 'GOOGL'),
            ('2019-08-23',  'MSFT'),
            ('2019-08-23',  'AAPL'),
            ('2019-08-23', 'GOOGL'),
            ('2019-08-23',  'AMZN'),
            ('2019-08-23',    'FB')],
           names=['date', 'name'], length=7105)

As we can see, the multiindex is a data structure in its own right. It has its own specific attributes, sequence of values, and methods.

The beauty of this is that the object can support very complex structures and hierarches in a self-contained way. Let's break this down one component at a time, starting with the "names" attribute. The `names` attribute returns a list of the labels that we're indexing by.

In [56]:
tech.index.names

FrozenList(['date', 'name'])

The next component is called `levels`, which contains a list of lists containing the range of values for each of the labels in the multiindex.

First, let's take a look at `nlevels`, or number of levels.

In [57]:
tech.index.nlevels

2

As we already know, this particular index has two levels (in our case "date" and "name").

Now let's look at the actual levels.

In [58]:
tech.index.levels

FrozenList([['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-13', '2014-01-14', '2014-01-15', '2014-01-16', '2014-01-17', '2014-01-21', '2014-01-22', '2014-01-23', '2014-01-24', '2014-01-27', '2014-01-28', '2014-01-29', '2014-01-30', '2014-01-31', '2014-02-03', '2014-02-04', '2014-02-05', '2014-02-06', '2014-02-07', '2014-02-10', '2014-02-11', '2014-02-12', '2014-02-13', '2014-02-14', '2014-02-18', '2014-02-19', '2014-02-20', '2014-02-21', '2014-02-24', '2014-02-25', '2014-02-26', '2014-02-27', '2014-02-28', '2014-03-03', '2014-03-04', '2014-03-05', '2014-03-06', '2014-03-07', '2014-03-10', '2014-03-11', '2014-03-12', '2014-03-13', '2014-03-14', '2014-03-17', '2014-03-18', '2014-03-19', '2014-03-20', '2014-03-21', '2014-03-24', '2014-03-25', '2014-03-26', '2014-03-27', '2014-03-28', '2014-03-31', '2014-04-01', '2014-04-02', '2014-04-03', '2014-04-04', '2014-04-07', '2014-04-08', '2014-04-09', '2014-04-10', '2014-04-11', '2014-0

Again, this returns a list of lists, which each list corresponding to a specific index level and containing all of the values within that index level. Since it's a list, we can examine each index individually.

In [59]:
tech.index.levels[0]

Index(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07', '2014-01-08',
       '2014-01-09', '2014-01-10', '2014-01-13', '2014-01-14', '2014-01-15',
       ...
       '2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16',
       '2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22', '2019-08-23'],
      dtype='object', name='date', length=1421)

In [60]:
tech.index.levels[1]

Index(['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT'], dtype='object', name='name')

A quick note - the highest level of the index (at the 0 position of the `levels` list), is the index level at the far left of the dataframe. As we move to the right, the levels increase. 

If we want to look at the length of each of our levels, we can use the `levshape` attribute.

In [61]:
tech.index.levshape

(1421, 5)

Thus, we have 1421 dates and 5 names (tickers).

Lastly, don't forget that multiindices are not just freely-floating sequences of labels. They represent a tight coupling of hierarchies of labels. An easy way to look at label combinations is to access the `values` attribute of the multiindex.

In [62]:
tech.index.values

array([('2014-01-02', 'FB'), ('2014-01-02', 'AAPL'),
       ('2014-01-02', 'GOOGL'), ..., ('2019-08-23', 'GOOGL'),
       ('2019-08-23', 'AMZN'), ('2019-08-23', 'FB')], dtype=object)

We see that this gives us a list of tuples, with each tuple containing two items - one date and one stock ticker. If we were to add a third dimension to our index, we would get a third value in each tuple. 

Essentially, the `index.values` attribute gives all existing combinations of the different index levels present in the dataframe. 

## Adding Another Level to a MultiIndex DataFrame

In this lecture, we'll be adding another level to our index, namely *volume_type*. 

The most intuitive way to do this is to use the `set_index()` method with the `append` parameter set to true.

In [63]:
tech.set_index('volume_type', append = True, inplace = True)

In [64]:
tech.head(10)

month  year  day  ...    high     low    volume
date       name  volume_type                    ...                          
2014-01-02 FB    medium           1  2014    2  ...   55.22   54.19  43257622
           AAPL  low              1  2014    2  ...   79.58   78.86   8398851
           GOOGL medium           1  2014    2  ...  558.88  554.13   1822719
           MSFT  medium           1  2014    2  ...   37.40   37.10  30643745
           AMZN  medium           1  2014    2  ...  399.36  394.02   2140246
2014-01-03 FB    medium           1  2014    3  ...   55.65   54.53  38287706
           GOOGL medium           1  2014    3  ...  558.47  552.47   1669229
           MSFT  medium           1  2014    3  ...   37.22   36.60  31134795
           AAPL  low              1  2014    3  ...   79.10   77.20  14043410
           AMZN  medium           1  2014    3  ...  402.71  396.22   2213512

[10 rows x 8 columns]

We see that "volume_type" is added as the third level of our index. Now let's check our index attributes.

In [65]:
tech.index.nlevels

3

In [66]:
tech.index.levels[2]

Index(['high', 'low', 'medium'], dtype='object', name='volume_type')

In [67]:
tech.index.values

array([('2014-01-02', 'FB', 'medium'), ('2014-01-02', 'AAPL', 'low'),
       ('2014-01-02', 'GOOGL', 'medium'), ...,
       ('2019-08-23', 'GOOGL', 'medium'),
       ('2019-08-23', 'AMZN', 'medium'), ('2019-08-23', 'FB', 'medium')],
      dtype=object)

As expected, the `values` attribute now consists of tuples of 3, which each tuple capture one attribute from each level of the multiindex.

Since we introduced another level, selecting by label will look a bit different from our two-index dataframes. For example, suppose we want to select data from high-volume trading days in January 2019. We can still use the `loc[]` indexer for this, but we'll need a longer tuple.

Our approach will be to slice all of the dates from January 2021 from the first level, all names from the second level, and specify only high-volume trading days from the third level. For this, we will use the `loc[]` indexer with the `slice()` object.

In [68]:
tech.loc[(slice("2019-01-01","2019-01-31"), slice(None), 'high'), :]

month  year  day  ...     high      low    volume
date       name volume_type                    ...                            
2019-01-03 AAPL high             1  2019    3  ...   145.72   142.00  91312195
2019-01-04 AMZN high             1  2019    4  ...  1594.00  1518.31   9182575
2019-01-08 AMZN high             1  2019    8  ...  1676.61  1616.61   8881428
2019-01-31 AMZN high             1  2019   31  ...  1736.41  1679.08  10910338
           FB   high             1  2019   31  ...   171.68   165.00  77233602

[5 rows x 8 columns]

As another example, let's grab all of the high-volume trading days irrespective of date or company. We can easily do this with the `xs()` method (although other approaches would work as well).

In [69]:
tech.xs(key = 'high', level = 2, drop_level = False)

month  year  day  ...     high      low    volume
date       name  volume_type                    ...                            
2014-01-06 FB    high             1  2014    6  ...    57.26    54.05  68974359
2014-01-07 FB    high             1  2014    7  ...    58.55    57.22  77329009
2014-01-08 FB    high             1  2014    8  ...    58.41    57.23  56800776
2014-01-09 FB    high             1  2014    9  ...    58.96    56.65  92349222
2014-01-13 FB    high             1  2014   13  ...    58.25    55.38  63106519
...                             ...   ...  ...  ...      ...      ...       ...
2019-04-30 GOOGL high             4  2019   30  ...  1200.98  1183.00   6658855
2019-06-03 FB    high             6  2019    3  ...   175.05   161.01  56059609
           AMZN  high             6  2019    3  ...  1766.29  1672.00   9098708
           GOOGL high             6  2019    3  ...  1067.00  1027.03   4844480
2019-07-26 GOOGL high             7  2019   26  ...  1268.39  1228.00   6060795

[375 rows x 8 columns]

Suppose we only cared about Facebook's high-volume days, we can add that in as well.

In [70]:
tech.xs(key = ('FB','high'), level = (1,2), drop_level = False)

,,,month,year,day,open,close,high,low,volume
date,name,volume_type,,,,,,,,
2014-01-06,FB,high,1,2014,6,54.39,57.20,57.26,54.05,68974359
2014-01-07,FB,high,1,2014,7,57.67,57.92,58.55,57.22,77329009
2014-01-08,FB,high,1,2014,8,57.59,58.23,58.41,57.23,56800776
2014-01-09,FB,high,1,2014,9,58.66,57.22,58.96,56.65,92349222
2014-01-13,FB,high,1,2014,13,57.89,55.91,58.25,55.38,63106519
...,...,...,...,...,...,...,...,...,...,...
2018-10-31,FB,high,10,2018,31,155.00,151.79,156.40,148.96,60101251
2018-12-19,FB,high,12,2018,19,141.21,133.24,144.91,132.50,57404894
2018-12-21,FB,high,12,2018,21,133.39,124.95,134.90,123.42,56901491


Just an interesting tidbit, according to our data, 115 out of 375 high-volume trading days have been driven by transaction of Facebook shares.

## Shuffling Levels with `swaplevel()` and `reorder_levels()`

What happens if we're unable with the order of our multiindices? When we added a new index in our last lecture, it was appended to the dataframe as the innermost (or lowest-level) index. What if instead we wanted it to be in a different order?

This is easily doable using the `swaplevel()` method, which swaps indices at two levels that we specify.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.swaplevel.html
* If given no arguments, the method by default swaps the two innermost levels.

In [71]:
tech.head()

month  year  day  ...    high     low    volume
date       name  volume_type                    ...                          
2014-01-02 FB    medium           1  2014    2  ...   55.22   54.19  43257622
           AAPL  low              1  2014    2  ...   79.58   78.86   8398851
           GOOGL medium           1  2014    2  ...  558.88  554.13   1822719
           MSFT  medium           1  2014    2  ...   37.40   37.10  30643745
           AMZN  medium           1  2014    2  ...  399.36  394.02   2140246

[5 rows x 8 columns]

In our example dataset, within our three indices we will swap "volume_type" and "name".

In [72]:
tech.swaplevel(i = 2, j = 1)

month  year  day  ...     high      low    volume
date       volume_type name                     ...                            
2014-01-02 medium      FB         1  2014    2  ...    55.22    54.19  43257622
           low         AAPL       1  2014    2  ...    79.58    78.86   8398851
           medium      GOOGL      1  2014    2  ...   558.88   554.13   1822719
                       MSFT       1  2014    2  ...    37.40    37.10  30643745
                       AMZN       1  2014    2  ...   399.36   394.02   2140246
...                             ...   ...  ...  ...      ...      ...       ...
2019-08-23 medium      MSFT       8  2019   23  ...   138.35   132.80  38515386
                       AAPL       8  2019   23  ...   212.05   201.00  46882843
                       GOOGL      8  2019   23  ...  1195.67  1150.00   1813141
                       AMZN       8  2019   23  ...  1804.90  1745.23   5277898
                       FB         8  2019   23  ...   183.13   176.66  17331221

[7105 rows x 8 columns]

Since "name" and "volume_type" are the two innermost indices, the default behavior will produce the same result.

In [73]:
tech.swaplevel()

month  year  day  ...     high      low    volume
date       volume_type name                     ...                            
2014-01-02 medium      FB         1  2014    2  ...    55.22    54.19  43257622
           low         AAPL       1  2014    2  ...    79.58    78.86   8398851
           medium      GOOGL      1  2014    2  ...   558.88   554.13   1822719
                       MSFT       1  2014    2  ...    37.40    37.10  30643745
                       AMZN       1  2014    2  ...   399.36   394.02   2140246
...                             ...   ...  ...  ...      ...      ...       ...
2019-08-23 medium      MSFT       8  2019   23  ...   138.35   132.80  38515386
                       AAPL       8  2019   23  ...   212.05   201.00  46882843
                       GOOGL      8  2019   23  ...  1195.67  1150.00   1813141
                       AMZN       8  2019   23  ...  1804.90  1745.23   5277898
                       FB         8  2019   23  ...   183.13   176.66  17331221

[7105 rows x 8 columns]

The method also works with label names instead of label positions.

In [74]:
tech.swaplevel(i = 'volume_type', j = 'name')

month  year  day  ...     high      low    volume
date       volume_type name                     ...                            
2014-01-02 medium      FB         1  2014    2  ...    55.22    54.19  43257622
           low         AAPL       1  2014    2  ...    79.58    78.86   8398851
           medium      GOOGL      1  2014    2  ...   558.88   554.13   1822719
                       MSFT       1  2014    2  ...    37.40    37.10  30643745
                       AMZN       1  2014    2  ...   399.36   394.02   2140246
...                             ...   ...  ...  ...      ...      ...       ...
2019-08-23 medium      MSFT       8  2019   23  ...   138.35   132.80  38515386
                       AAPL       8  2019   23  ...   212.05   201.00  46882843
                       GOOGL      8  2019   23  ...  1195.67  1150.00   1813141
                       AMZN       8  2019   23  ...  1804.90  1745.23   5277898
                       FB         8  2019   23  ...   183.13   176.66  17331221

[7105 rows x 8 columns]

Unfortunately, this method does not include an `inplace` parameter. Thus if we want to make our changes permanent, we have to reassign the variable to the new dataframe. Let's go ahead and do that - in this case it works nicely because the "least variable" index types are at the higher levels. Thus, as we work "inward" in our indices, they have increasing variability.

In [75]:
tech = tech.swaplevel(i = 'volume_type', j = 'name')

In [76]:
tech.head()

month  year  day  ...    high     low    volume
date       volume_type name                     ...                          
2014-01-02 medium      FB         1  2014    2  ...   55.22   54.19  43257622
           low         AAPL       1  2014    2  ...   79.58   78.86   8398851
           medium      GOOGL      1  2014    2  ...  558.88  554.13   1822719
                       MSFT       1  2014    2  ...   37.40   37.10  30643745
                       AMZN       1  2014    2  ...  399.36  394.02   2140246

[5 rows x 8 columns]

In Pandas, there is also another more powerful method that allows you to do more extensive reorders with more than two indices. This is the `reorder_levels()` method, which allows you to express a new order for the multiindex all in one go. All you need to do is pass in a list of the indices declared in the order in which you want them positioned.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reorder_levels.html

In [77]:
tech.reorder_levels([2, 0, 1])

,,,month,year,day,open,close,high,low,volume
name,date,volume_type,,,,,,,,
FB,2014-01-02,medium,1,2014,2,54.86,54.71,55.22,54.19,43257622
AAPL,2014-01-02,low,1,2014,2,79.38,79.02,79.58,78.86,8398851
GOOGL,2014-01-02,medium,1,2014,2,557.73,556.56,558.88,554.13,1822719
MSFT,2014-01-02,medium,1,2014,2,37.35,37.16,37.40,37.10,30643745
AMZN,2014-01-02,medium,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...,...,...
MSFT,2019-08-23,medium,8,2019,23,137.19,133.39,138.35,132.80,38515386
AAPL,2019-08-23,medium,8,2019,23,209.43,202.64,212.05,201.00,46882843
GOOGL,2019-08-23,medium,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


Instead of calling the reorder on the enter dataframe, we also have the option of calling it on the index object itself. This will return the reordered multiindex.

In [78]:
tech.index.reorder_levels([2, 0, 1])

MultiIndex([(   'FB', '2014-01-02', 'medium'),
            ( 'AAPL', '2014-01-02',    'low'),
            ('GOOGL', '2014-01-02', 'medium'),
            ( 'MSFT', '2014-01-02', 'medium'),
            ( 'AMZN', '2014-01-02', 'medium'),
            (   'FB', '2014-01-03', 'medium'),
            ('GOOGL', '2014-01-03', 'medium'),
            ( 'MSFT', '2014-01-03', 'medium'),
            ( 'AAPL', '2014-01-03',    'low'),
            ( 'AMZN', '2014-01-03', 'medium'),
            ...
            ( 'MSFT', '2019-08-22', 'medium'),
            (   'FB', '2019-08-22',    'low'),
            ( 'AMZN', '2019-08-22', 'medium'),
            ( 'AAPL', '2019-08-22', 'medium'),
            ('GOOGL', '2019-08-22',    'low'),
            ( 'MSFT', '2019-08-23', 'medium'),
            ( 'AAPL', '2019-08-23', 'medium'),
            ('GOOGL', '2019-08-23', 'medium'),
            ( 'AMZN', '2019-08-23', 'medium'),
            (   'FB', '2019-08-23', 'medium')],
           names=['name', 'date', 'volume_t

## Removing MultiIndex Levels with `droplevel()`

In previous lectures we worked on adding new levels to our multiindex. But what about removing levels?

Pandas recently introduced a method that makes this very easy. The `droplevel()` method allows you to drop a level from your multiindex.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.droplevel.html
* The levels to be dropped can be identified by index position or by name
* This method discards the index that is removed - there is no way to keep that level around using this method.

In [79]:
tech.head()

month  year  day  ...    high     low    volume
date       volume_type name                     ...                          
2014-01-02 medium      FB         1  2014    2  ...   55.22   54.19  43257622
           low         AAPL       1  2014    2  ...   79.58   78.86   8398851
           medium      GOOGL      1  2014    2  ...  558.88  554.13   1822719
                       MSFT       1  2014    2  ...   37.40   37.10  30643745
                       AMZN       1  2014    2  ...  399.36  394.02   2140246

[5 rows x 8 columns]

In [80]:
tech.droplevel(1)

month  year  day  ...     high      low    volume
date       name                     ...                            
2014-01-02 FB         1  2014    2  ...    55.22    54.19  43257622
           AAPL       1  2014    2  ...    79.58    78.86   8398851
           GOOGL      1  2014    2  ...   558.88   554.13   1822719
           MSFT       1  2014    2  ...    37.40    37.10  30643745
           AMZN       1  2014    2  ...   399.36   394.02   2140246
...                 ...   ...  ...  ...      ...      ...       ...
2019-08-23 MSFT       8  2019   23  ...   138.35   132.80  38515386
           AAPL       8  2019   23  ...   212.05   201.00  46882843
           GOOGL      8  2019   23  ...  1195.67  1150.00   1813141
           AMZN       8  2019   23  ...  1804.90  1745.23   5277898
           FB         8  2019   23  ...   183.13   176.66  17331221

[7105 rows x 8 columns]

The instructor prefers using `reset_index()` because it's more powerful and flexible. It allows us to get rid of a level (or multiple levels) if we want to, and simultaneously restores that level (or levels) as a column in the dataframe.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
* If you wish, you can discared the removed index by setting the `drop` parameter to `False`. 
* Whereas a single-index dataframe will reset to the default index, a multi-index dataframe will simply remove the specified index and keep the remainders.

In [81]:
tech.reset_index(1)

volume_type  month  year  ...     high      low    volume
date       name                            ...                            
2014-01-02 FB         medium      1  2014  ...    55.22    54.19  43257622
           AAPL          low      1  2014  ...    79.58    78.86   8398851
           GOOGL      medium      1  2014  ...   558.88   554.13   1822719
           MSFT       medium      1  2014  ...    37.40    37.10  30643745
           AMZN       medium      1  2014  ...   399.36   394.02   2140246
...                      ...    ...   ...  ...      ...      ...       ...
2019-08-23 MSFT       medium      8  2019  ...   138.35   132.80  38515386
           AAPL       medium      8  2019  ...   212.05   201.00  46882843
           GOOGL      medium      8  2019  ...  1195.67  1150.00   1813141
           AMZN       medium      8  2019  ...  1804.90  1745.23   5277898
           FB         medium      8  2019  ...   183.13   176.66  17331221

[7105 rows x 9 columns]

What do we do if we want to remove multiple levels from our index?

One way is to call `drop_level()` or `reset_index()` multiple times. Thankfully, both of these methods accept lists of level names (or level positions) so that we can remove more than one at a time.

In [82]:
tech.droplevel(['volume_type', 'name'])

,month,year,day,open,close,high,low,volume
date,,,,,,,,
2014-01-02,1,2014,2,54.86,54.71,55.22,54.19,43257622
2014-01-02,1,2014,2,79.38,79.02,79.58,78.86,8398851
2014-01-02,1,2014,2,557.73,556.56,558.88,554.13,1822719
2014-01-02,1,2014,2,37.35,37.16,37.40,37.10,30643745
2014-01-02,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...
2019-08-23,8,2019,23,137.19,133.39,138.35,132.80,38515386
2019-08-23,8,2019,23,209.43,202.64,212.05,201.00,46882843
2019-08-23,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


In [83]:
tech.droplevel([1, 2])

,month,year,day,open,close,high,low,volume
date,,,,,,,,
2014-01-02,1,2014,2,54.86,54.71,55.22,54.19,43257622
2014-01-02,1,2014,2,79.38,79.02,79.58,78.86,8398851
2014-01-02,1,2014,2,557.73,556.56,558.88,554.13,1822719
2014-01-02,1,2014,2,37.35,37.16,37.40,37.10,30643745
2014-01-02,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...
2019-08-23,8,2019,23,137.19,133.39,138.35,132.80,38515386
2019-08-23,8,2019,23,209.43,202.64,212.05,201.00,46882843
2019-08-23,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


In [84]:
tech.reset_index(['volume_type','name'], drop = True)

,month,year,day,open,close,high,low,volume
date,,,,,,,,
2014-01-02,1,2014,2,54.86,54.71,55.22,54.19,43257622
2014-01-02,1,2014,2,79.38,79.02,79.58,78.86,8398851
2014-01-02,1,2014,2,557.73,556.56,558.88,554.13,1822719
2014-01-02,1,2014,2,37.35,37.16,37.40,37.10,30643745
2014-01-02,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...
2019-08-23,8,2019,23,137.19,133.39,138.35,132.80,38515386
2019-08-23,8,2019,23,209.43,202.64,212.05,201.00,46882843
2019-08-23,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141


It's also worth noting that `reset_index()` is useful from eliminating our index entirely and resetting it to the standard range index. All of the existing indices will be moved into the dataframe as columns, unless the `drop` parameter is set to `False`.

In [85]:
tech.reset_index()

,date,volume_type,name,month,year,day,open,close,high,low,volume
0,2014-01-02,medium,FB,1,2014,2,54.86,54.71,55.22,54.19,43257622
1,2014-01-02,low,AAPL,1,2014,2,79.38,79.02,79.58,78.86,8398851
2,2014-01-02,medium,GOOGL,1,2014,2,557.73,556.56,558.88,554.13,1822719
3,2014-01-02,medium,MSFT,1,2014,2,37.35,37.16,37.40,37.10,30643745
4,2014-01-02,medium,AMZN,1,2014,2,398.80,397.97,399.36,394.02,2140246
...,...,...,...,...,...,...,...,...,...,...,...
7100,2019-08-23,medium,MSFT,8,2019,23,137.19,133.39,138.35,132.80,38515386
7101,2019-08-23,medium,AAPL,8,2019,23,209.43,202.64,212.05,201.00,46882843
7102,2019-08-23,medium,GOOGL,8,2019,23,1185.17,1153.58,1195.67,1150.00,1813141
7103,2019-08-23,medium,AMZN,8,2019,23,1793.03,1749.62,1804.90,1745.23,5277898


## Sorting MultiIndex DataFrames with `sort_index()`

Consider this slice of our dataframe. where we select the opening, closing, high, and low prices for Apple from Jan 2, 2014 through April 2, 2014 for all volume types.

In [86]:
tech.loc[(slice('2014-01-02','2014-04-02'), slice(None), 'AAPL'), 'open':'low']

,,,open,close,high,low
date,volume_type,name,,,,
2014-01-02,low,AAPL,79.38,79.02,79.58,78.86
2014-01-03,low,AAPL,79.00,77.28,79.10,77.20
2014-01-06,low,AAPL,76.78,77.70,78.11,76.23
2014-01-07,low,AAPL,77.76,77.15,77.99,76.85
2014-01-08,low,AAPL,76.97,77.64,77.94,76.96
...,...,...,...,...,...,...
2014-03-27,low,AAPL,77.11,76.78,77.36,76.45
2014-03-28,low,AAPL,76.82,76.69,76.99,76.32
2014-03-31,low,AAPL,77.03,76.68,77.26,76.56


What if we wanted to select not just Apple, but a slice going from Apple to Facebook. How would we do that?

Let's first try using a `slice()` object to slice from *AAPL* to *FB*. This does not work - Pandas returns an unsorted index error.

In [95]:
## This generates an UnsortedIndexError
# tech.loc[(slice('2014-01-02','2014-04-02'), slice(None), slice('AAPL','FB')), 'open':'low']

The reason for this is that it's not very meaningful to talk about slicing along a sequence of values that is not sorted or does not have an intrinsic order. Pandas wants us to sort our index before slicing it.
* Note that the highest level index (the date) is already sorted, so we can easily slice on that

So how do we go about sorting our indexes? We can do this using the `sort_index()` method. If we run this with no arguments, we see that every level becomes nice and sorted. By default, the sort is in ascending order.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html

In [92]:
tech.sort_index()

month  year  day  ...     high      low    volume
date       volume_type name                     ...                            
2014-01-02 low         AAPL       1  2014    2  ...    79.58    78.86   8398851
           medium      AMZN       1  2014    2  ...   399.36   394.02   2140246
                       FB         1  2014    2  ...    55.22    54.19  43257622
                       GOOGL      1  2014    2  ...   558.88   554.13   1822719
                       MSFT       1  2014    2  ...    37.40    37.10  30643745
...                             ...   ...  ...  ...      ...      ...       ...
2019-08-23 medium      AAPL       8  2019   23  ...   212.05   201.00  46882843
                       AMZN       8  2019   23  ...  1804.90  1745.23   5277898
                       FB         8  2019   23  ...   183.13   176.66  17331221
                       GOOGL      8  2019   23  ...  1195.67  1150.00   1813141
                       MSFT       8  2019   23  ...   138.35   132.80  38515386

[7105 rows x 8 columns]

Let's make this permanent by setting the `inplace` parameter to `True`

In [93]:
tech.sort_index(inplace = True)

Now that our index is sorted, we can proceed to slice on the "name" level of the index.

In [94]:
tech.loc[(slice('2014-01-02','2014-04-02'), slice(None), slice('AAPL','FB')), 'open':'low']

open   close    high     low
date       volume_type name                                
2014-01-02 low         AAPL   79.38   79.02   79.58   78.86
           medium      AMZN  398.80  397.97  399.36  394.02
                       FB     54.86   54.71   55.22   54.19
2014-01-03 low         AAPL   79.00   77.28   79.10   77.20
           medium      AMZN  398.29  396.44  402.71  396.22
...                             ...     ...     ...     ...
2014-04-01 low         AAPL   76.84   77.38   77.41   76.68
           medium      AMZN  338.35  342.99  344.43  338.00
2014-04-02 high        FB     63.24   62.72   63.91   62.21
           low         AAPL   77.48   77.51   77.64   77.18
           medium      AMZN  345.99  341.96  348.30  340.38

[189 rows x 4 columns]

In this lecture we sorted the index to address an error. However in general, the instructor recommends always sorting the index, especially when working with multiindex dataframes. Why?
* Sorting indexes improves retrieval performance, which becomes significant for large dataframs or for frequent retrieval activities
* Enables slicing syntax. You can only slice on sorted indexes
* Overall, it is a good practice when working with tabular data representations, including Pansas, Excel, SQL, etc.

For multiindex dataframes, we can also fine-turn the sort using the `level` parameter. For instance, if we wanted to sort one level in descending order another in ascending order, we can specify that in the `sort_index()` method. We must also pass a list of booleans to the `ascending` parameter so that Pandas knows which level to sort by ascending and descending order.
* Remember that if we don't customize the sort, then each level of the multiindex is sorted in ascending order by default

In [96]:
tech.sort_index(level=(0, 2), ascending=[False, True])

month  year  day  ...     high      low    volume
date       volume_type name                     ...                            
2019-08-23 medium      AAPL       8  2019   23  ...   212.05   201.00  46882843
                       AMZN       8  2019   23  ...  1804.90  1745.23   5277898
                       FB         8  2019   23  ...   183.13   176.66  17331221
                       GOOGL      8  2019   23  ...  1195.67  1150.00   1813141
                       MSFT       8  2019   23  ...   138.35   132.80  38515386
...                             ...   ...  ...  ...      ...      ...       ...
2014-01-02 low         AAPL       1  2014    2  ...    79.58    78.86   8398851
           medium      AMZN       1  2014    2  ...   399.36   394.02   2140246
                       FB         1  2014    2  ...    55.22    54.19  43257622
                       GOOGL      1  2014    2  ...   558.88   554.13   1822719
                       MSFT       1  2014    2  ...    37.40    37.10  30643745

[7105 rows x 8 columns]